<a href="https://colab.research.google.com/github/predatorx7/boring/blob/master/pyai/5_B_TicTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from enum import Enum


class Sign(Enum):
    x = 'x'
    o = 'o'


class Player(Enum):
    MAX = +1
    MIN = -1

class Board:
    def __init__(self, max_plays_as: Sign):
        assert isinstance(max_plays_as, Sign)
        self.max_sign = max_plays_as
        if self.max_sign == Sign.x:
            self.min_sign = Sign.o
        else:
            self.min_sign = Sign.x
        self.reset()

    def reset(self):
        self.board = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        self.size = len(self.board)
        self.winner = None

    def set_move(self, atRow, atColumn, player: Player):
        assert (atRow, atColumn) in self.possible_moves(
        ), "This move is not possible"
        assert isinstance(player, Player)
        symbol = player.value
        self.board[atRow][atColumn] = symbol

    def possible_moves(self):
        moves = []
        for i in range(self.size):
            for j in range(self.size):
                tile = self.board[i][j]
                if tile == 0:
                    moves.append((i, j))
        return tuple(moves)

    def depth(self):
        _depth = 0
        for i in self.board:
            for j in i:
                if j == 0:
                    _depth += 1
        return _depth

    def has_empty_cells(self):
        for row in self.board:
            for cell in row:
                if cell == 0:
                    return True
        return False

    def get_state(self):
        states = []
        pdiagonal, sdiagonal = [], []
        for x in range(self.size):
            row, column = [], []
            for y in range(self.size):
                tileXY = self.board[x][y]
                tileYX = self.board[y][x]
                if (x == y):
                    pdiagonal.append(tileXY)
                if ((x + y) == (self.size - 1)):
                    sdiagonal.append(tileXY)
                row.append(tileXY)
                column.append(tileYX)
            states.append(row)
            states.append(column)
        states.append(pdiagonal)
        states.append(sdiagonal)
        return states

    def who_won(self):
        state = self.get_state()
        self.winner = None
        if [Player.MAX.value for i in range(self.size)] in state:
            self.winner = Player.MAX
        elif [Player.MIN.value for i in range(self.size)] in state:
            self.winner = Player.MIN
        return self.winner

    def is_game_over(self):
        if self.who_won() != None:
            return True
        return not self.has_empty_cells()

    def copy(self):
        board = Board(self.max_sign)
        board.board = []
        for i in self.board:
            board.board.append(i.copy())
        board.size = self.size
        return board

    def __str__(self):
        output = ''
        for row in self.board:
            for tile in row:
                if tile == Player.MAX.value:
                    output += f' {self.max_sign.value}'
                elif tile == Player.MIN.value:
                    output += f' {self.min_sign.value}'
                else:
                    output += ' _'
            output += '\n'
        return output

In [2]:
from math import inf

def max(a: float, a_move: tuple, b: float, b_move: tuple):
    assert (a != None and b != None)
    if (a > b):
        return a, a_move
    return b, b_move


def min(a: float, a_move: tuple, b: float, b_move: tuple):
    assert (a != None and b != None)
    if (a < b):
        return a, a_move
    return b, b_move

class Counter:
    def __init__(this, count = None):
        this.__count = 0 if count == None else count
    def get_count(this):
        return this.__count
    def increment(this):
        this.__count += 1
    def __str__(this):
        return str(this.__count)

def minmax(board: Board, is_maximizer: bool, counter: Counter = None):
    """```pseudo-code
    function minmax(node, depth, α, β, maximizingPlayer) is
        if depth = 0 or node is a terminal node then
            return the heuristic value of node
        if maximizingPlayer then
            best_value := −∞
            for each child of node do
                value := max(value, minmax(child, depth − 1, FALSE))
                best_value := max(best_value, value)
            return best_value
        else
            best_value := +∞
            for each child of node do
                value := min(value, minmax(child, depth − 1, TRUE))
                best_value := min(best_value, value)
            return best_value
    ```
    """
    if counter != None: counter.increment()
    if (board.is_game_over()):
        if board.winner != None:
            return board.winner.value, ()
        return 0, ()

    best_value = None
    best_move = ()
    player = None

    if (is_maximizer):
        player = Player.MAX
        best_value = -inf
    else:
        player = Player.MIN
        best_value = inf

    for r, c in board.possible_moves():
        board.board[r][c] = player.value
        value, _ = minmax(board, not is_maximizer, counter=counter)
        board.board[r][c] = 0
        new_move = (r, c)
        if is_maximizer:
            best_value, best_move = max(value, new_move, best_value, best_move)
        else:
            best_value, best_move = min(value, new_move, best_value, best_move)

    if (best_value == None):
        raise TypeError('value should not be None')
    return best_value, best_move

In [3]:
import os

def isstandard():
    try:
        shell = get_ipython().__class__.__name__
        return False  # Other type (?)
    except NameError:
        return True      # Probably standard Python interpreter

def clear():
    if not isstandard():
        from IPython.display import clear_output # ONLY FOR JUPYTER NOTEBOOKS
        clear_output()
    else:
        os.system('cls' if os.name == 'nt' else 'clear')

def valid_move(board: Board, x, y):
    """
    A move is valid if the chosen cell is empty
    """
    return (x, y) in board.possible_moves()

def game_over(board: Board):
    return board.is_game_over()

def winner(board: Board):
    return board.winner

def set_move(board: Board, x, y, player: Player):
    if valid_move(board, x, y):
        board.set_move(x, y, player)
        return True
    else:
        return False

In [4]:
from random import choice
import time

def human_turn(board: Board):
    if game_over(board):
        return

    clear() # clear console
    
    print()
    print(board)
    print(f"\nIt's your turn [{board.max_sign.value}].\n")

    move = None
    moves = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1), 9: (2, 2),
    }
    print('MOVES:\n 1 2 3\n 4 5 6\n 7 8 9')
    while not move in moves:
        try:
          move = int(input('Use numpad (1..9): '))
          x, y = moves[move]
          did_move = set_move(board, x, y, Player.MAX)
          if not did_move:
              print('Wrong move')
              move = None
        except (KeyError, ValueError):
          print('Illegal move')
          move = None

def ai_turn(board: Board):
    if game_over(board):
        return

    print(f"\nComputer's turn [{board.min_sign.value}].\n")

    x = y = None

    if board.depth() == 9:
        x = choice((0, 1, 2))
        y = choice((0, 1, 2))
    else:
        print("Computer is thinking.. \n")
        _, optimal_move = minmax(board, False)
        x, y = optimal_move
    board.set_move(x, y, Player.MIN)
    time.sleep(1) # give a pause

def winner(board: Board):
    "Warning: This does not evaluate results. Only use if game is over."
    
    clear() # clear console
    
    print()
    print(board)
    if board.winner == None:
        print("It's a Tie")
        return
    if board.winner == Player.MAX:
        print(f"You [{board.max_sign.value}] won!")
    else:
        print(f"You lost. The computer [{board.min_sign.value}] won!")

In [5]:
def main():
    player_sym = None
    reply = input("Choose X or O: ").upper()
    while not reply in ['X', 'O']:
        print("Wrong choice")
        reply = input("Choose X or O: ").upper()
    if reply == 'X':
        player_sym = Sign.x
    else:
        player_sym = Sign.o
    
    board = Board(player_sym)

    human_first = player_sym == Sign.x
    first = True

    while not game_over(board):
        human_play = human_first + first
        if human_play in [0, 2]:
            human_turn(board)
        else:
            ai_turn(board)
        first = not first

    winner(board)

In [6]:
main()


 x o x
 o o x
 x x o

It's a Tie
